In [154]:
import numpy as np
import matplotlib.pyplot as plt
import math
import pandas as pd
import os
import shutil
from Cyclist_env_RDA_2nano import cyclist_env
from time import time
from scipy.fft import fft, ifft
from scipy.linalg import dft

import torch
import pandas as pd
import torch.nn.functional as F
from scipy.optimize import linear_sum_assignment
from scipy.ndimage import maximum_filter
from scipy.optimize import minimize_scalar

In [155]:
def Radar_setting():
    ###### symbol time & carrier frequency ######
    T_symbol = 5.575 * 1e-6              # symbol duration, with CP time
    T_OFDM = 5.2125 * 1e-6
    f_carrier = 28 * 1e+9
    Tc = 2.545*1e-9                      # sampling time

    ###### tx/rx ######
    N_ant = 16                       # the number of antennas
    # BW = 1.966080e+9                      # chirp bandwidth
    BW = 400*1e+6                    # chirp bandwidth
    BW_sub = BW/N_ant
    N_sample = int(np.floor(T_symbol/Tc))           # the number of samples of single chirp
    rx_sample = 813 + N_sample

    ###### Radar setting ######

    mu = BW_sub/T_symbol * 0.98
    l_speed = 299792458

    lam = l_speed/f_carrier
    env = cyclist_env(f_carrier, N_ant, BW, BW_sub, N_sample, rx_sample, Tc, mu, l_speed)
    return T_symbol,f_carrier,Tc,N_ant,BW,BW_sub,N_sample,rx_sample, mu, l_speed, lam, env

In [230]:
def Sim_Setting():
    # シミュレーション設定
    p_bs = np.array([250, -18, 50])                         #基地局の位置

    # 各車両の数
    num_cy = 1
    num_ve = 1
    num_rp = 0

    
    # シミュレーション範囲の設定
    start_cy_idx = 150
    start_ve_idx = 100
    end_cy_idx = 240
    end_ve_idx =250
    
    """
    #実験用
    start_cy_idx = 240
    start_ve_idx = 250

    end_cy_idx = 240
    end_ve_idx = 250
    """

    cy_interval = 3
    ve_interval = 5

    cy_v_value = 6
    ve_v_value = 10
    
    return p_bs, num_cy, num_ve, num_rp, start_cy_idx, start_ve_idx, end_cy_idx, end_ve_idx, cy_v_value, ve_v_value, cy_interval, ve_interval

In [157]:
def syntax_check(num, idx, y_value, velocity_value, x_pos):
    idx_list = np.zeros(num, dtype=np.int32)
    position, velocity = np.zeros((num, 3)), np.zeros((num, 3))
    i=0
    s = set()
    while i < num:
        # 初期値における距離間隔を保証するための処理
        if idx+10 in s or idx-10 in s or idx in s: continue
        for j in range(-20,20): s.add(idx+j)
        
        idx_list[i] = idx
        position[i] = np.array([x_pos[idx], y_value, 1])
        # v_cy[i] = np.array([np.random.rand(1)[0]*5.56, 0, 0])
        velocity[i] = np.array([velocity_value, 0, 0])
        
        i+=1
    return idx_list, position, velocity

In [158]:
def get_snapshot_data(cy_idx_target, ve_idx_target,x_pos, p_bs, tx, cy_v_value, ve_v_value, num_cy, num_ve, num_rp, sym_duration, N_trans,Tc, env):
    # Cyclistについて
    cy_idx = np.zeros(num_cy, dtype=np.int32)
    p_cy, v_cy = np.zeros((num_cy, 3)), np.zeros((num_cy, 3))
    # 範囲外アクセスを防ぐための処理
    cy_idx, p_cy, v_cy = syntax_check(num_cy, cy_idx_target, 15, cy_v_value, x_pos)
    n_cy = len(p_cy)
    
    # Vehicleについて
    ve_idx = np.zeros(num_ve, dtype=np.int32)
    p_ve, v_ve = np.zeros((num_ve, 3)), np.zeros((num_ve, 3))
    # 範囲外アクセスを防ぐための処理
    ve_idx, p_ve, v_ve = syntax_check(num_ve, ve_idx_target, 7.5, ve_v_value, x_pos)
    n_ve = len(p_ve)

    # ramppostについて
    p_rp, v_rp = np.zeros((num_rp, 3)), np.zeros((num_rp, 3))
    n_rp = len(p_rp)
    
    # csvファイルの読み込み
    cy_idx += 1
    ve_idx += 1

    P_rx_cy, tstemp_rx_cy, phase_cy, P_rx_ve, tstemp_rx_ve, phase_ve, P_rx_rp, tstemp_rx_rp, phase_rp = [], [], [], [], [], [], [], [], []
    # 3番目のデータから読み込み開始（最初の数個はノイズのみのため）
    for i in range(len(cy_idx)):
        df = pd.read_csv("./ped/2.545nano/far/complex-impulse-response-Run{:04d}-Sensor_0_Tx_0_to_Rx_0.csv".format(cy_idx[i]))
        tstemp_rx_cy.append(np.round((df["Time (s)"]/Tc)).values[3:])
        P_rx_cy.append(df["| Total Complex Impulse Response total | (W)"].values[3:])
        phase_cy.append(df['Phase( Total Complex Impulse Response total ) (rad)'].values[3:])
    for i in range(len(ve_idx)):
        df = pd.read_csv("./vehicle/2.545nano/far/complex-impulse-response-Run{:04d}-Sensor_0_Tx_0_to_Rx_0.csv".format(ve_idx[i]))
        tstemp_rx_ve.append(np.round((df["Time (s)"].values[3:]/Tc)))
        P_rx_ve.append(df["| Total Complex Impulse Response total | (W)"].values[3:])
        phase_ve.append(df['Phase( Total Complex Impulse Response total ) (rad)'].values[3:])
    # 2.545のほうを使う！
    for i in range(2,3):
        df = pd.read_csv("./Ramposts/2.545nano/"+str(i)+"/complex-impulse-response-Run0001-Sensor_0_Tx_0_to_Rx_0.csv")
        tstemp_rx_rp.append(np.round((df["Time (s)"].values[5:]/Tc)))
        P_rx_rp.append(df["| Total Complex Impulse Response total | (W)"].values[5:])
        phase_rp.append(df['Phase( Total Complex Impulse Response total ) (rad)'].values[5:])
    phys_quantities = env.phys_quantities(p_bs, p_cy, v_cy, n_cy, p_ve, v_ve, n_ve, p_rp, v_rp, n_rp)
    #print(phys_quantities["relative_velocity"]["cyclists"])
    real_cy_coordinates = []
    real_vel_coordinates = []
    real_rp_coordinates = []

    for i in range(len(p_cy)):
        real_cy_coordinates.append(np.array([p_cy[i][0], p_cy[i][1]]))
    for i in range(len(p_ve)):
        real_vel_coordinates.append(np.array([p_ve[i][0], p_ve[i][1]]))
    for i in range(len(p_rp)):
        real_rp_coordinates.append(np.array([p_rp[i][0], p_rp[i][1]]))
    
        
    ######### We need some functions here. it is like this. read the csv. this outputs the time stemp and corresponding rx value.
    # P_rx_cy_dB = -114
    #P_rx_cy_dB = -111
    #P_rx_ve_dB = -114
    #P_rx_cy_dB = 0
    #P_rx_ve_dB = 0
    
    P_N_dB = -87.98

    """
    # =================================================================
    # 【実験用追加コード】自転車の信号強度ドーピング
    # =================================================================
    # 目的: 強度差が原因であることを証明するため、自転車の信号を人工的に強める
    # 設定: +15dB (約31.6倍) して、車(-7.8dB)と自転車(-25dB)の差を埋める
    
    target_gain_dB = 15.0  # ここで増幅量を調整 (12~20dB推奨)
    gain_linear = 10 ** (target_gain_dB / 10.0) # dBを倍率に変換
    
    print(f"\n[Experimental] Injecting +{target_gain_dB}dB gain to Cyclist signal...")
    
    # P_rx_cy はリスト形式 ([array(..), array(..)]) なのでループで処理
    for i in range(len(P_rx_cy)):
        original_power = 10 * np.log10(np.mean(P_rx_cy[i]) + 1e-20) # 元の平均強度(参考)
        
        # パワー(W)を定数倍する
        P_rx_cy[i] = P_rx_cy[i] * gain_linear
        
        new_power = 10 * np.log10(np.mean(P_rx_cy[i]) + 1e-20) # 後の平均強度(参考)
        print(f"  -> Cyclist {i}: {original_power:.1f}dB => {new_power:.1f}dB")

    # =================================================================
    """

    """
    # =================================================================
    # 【実験用追加コード】自動車の信号強度ドーピング
    # =================================================================
    # 目的: 強度差が原因であることを証明するため、自動車の信号を人工的に強める
    # 設定: -15dB (約31.6倍) して、車(-7.8dB)と自転車(-25dB)の差を埋める
    
    target_gain_dB = -15.0  # ここで増幅量を調整 (12~20dB推奨)
    gain_linear = 10 ** (target_gain_dB / 10.0) # dBを倍率に変換
    
    print(f"\n[Experimental] Injecting {target_gain_dB}dB gain to Vehicle signal...")
    
    # P_rx_ve はリスト形式 ([array(..), array(..)]) なのでループで処理
    for i in range(len(P_rx_ve)):
        original_power = 10 * np.log10(np.mean(P_rx_ve[i]) + 1e-20) # 元の平均強度(参考)
        
        # パワー(W)を定数倍する
        P_rx_ve[i] = P_rx_ve[i] * gain_linear
        
        new_power = 10 * np.log10(np.mean(P_rx_ve[i]) + 1e-20) # 後の平均強度(参考)
        print(f"  -> Vehicle {i}: {original_power:.1f}dB => {new_power:.1f}dB")

    # =================================================================
    """
    Y = env.rx_multiple(tx, P_rx_cy, tstemp_rx_cy, phase_cy, P_rx_ve, tstemp_rx_ve, phase_ve, P_rx_rp, tstemp_rx_rp, phase_rp, P_N_dB, sym_duration, N_trans)
    
    return Y, phys_quantities, real_cy_coordinates, real_vel_coordinates, real_rp_coordinates

In [159]:
def derectory_deletor(directory_path):
    try:
        # ディレクトリ内のすべてのエントリ（ファイルとフォルダ）を取得
        for entry in os.listdir(directory_path):
            full_path = os.path.join(directory_path, entry)

            # ファイルかどうかをチェック
            if os.path.isfile(full_path):
                # ファイルを削除
                os.remove(full_path)
            
            # サブディレクトリの場合はスキップ
            elif os.path.isdir(full_path):
                print(f"⏭️ サブディレクトリはスキップしました: {full_path}")
                # サブディレクトリ内のファイルも削除したい場合は、
                # ここで再帰的にこの関数を呼び出すなどの処理が必要です
                # (例: delete_all_files_in_directory(full_path))
    except FileNotFoundError:
        print(f"エラー: ディレクトリが見つかりません - {directory_path}")
    except Exception as e:
        print(f"ファイル削除中にエラーが発生しました: {e}")
    return

In [207]:
def MUSIC_method_improved(Y, N_ant, num_signals):
    '''
    修正前
    Y_music = np.mean(Y, axis=0)
    R_yy = Y_music@np.conjugate(np.transpose(Y_music))
    '''
    # 修正後: 
    # チャープ軸(0)とサンプル軸(2)の両方を「スナップショット」として扱います。
    # (N_trans, N_ant, N_sample) -> (N_ant, N_trans * N_sample) に変形
    # これにより、N_trans(チャープ数)が増えるほど、スナップショット数が増え、
    # 共分散行列 R_yy の推定精度が向上します。
    
    N_trans, N_ant_data, N_sample = Y.shape
    # データを (アンテナ数 x 全サンプル数) の2次元行列に変形
    Y_music = Y.transpose(1, 0, 2).reshape(N_ant_data, -1)
    
    # 共分散行列の計算
    # (行列サイズは アンテナ数 x アンテナ数 のまま変わらないので計算負荷は軽微です)
    R_yy = (Y_music @ np.conjugate(Y_music.T)) / Y_music.shape[1]
    
    # 1.固有値分解
    eig_val, eig_vec = np.linalg.eig(R_yy)

    # 2.固有値をソートして雑音部分空間を取得
    idx = np.abs(eig_val).argsort()[::-1]
    # インデックスによって固有値と固有ベクトルの対応関係を維持しつつソート
    eig_val = eig_val[idx]
    eig_vec = eig_vec[:, idx]
    # 雑音部分空間Uを取得

    # num_signalsを固有値から推定?
    
    # デバッグ用
    num_signals = 5
    
    U = eig_vec[:, num_signals:]

    # 3.雑音部分空間とvalごとのステアリングベクトルの内積を計算
    resp = []
    argm = (np.arange(1800)-900)/100
    for val in argm:
        # ステアリングベクトルの計算
        stv = stevec(N_ant, val*np.pi/180)
        # 雑音部分空間との内積
        p = stv.T@U
        # L2ノルムの二乗を計算(pp^H)
        pp = p*np.conjugate(np.transpose(p))
        # A1は行列を1次元配列に変換するメソッド
        # respに逆数を追加
        # 分子は角度推定の上では不要なので省略している
        resp.append(1/(np.abs(pp)**2).A1)

    # 4.ピーク検出
    M = np.max(resp)
    
    # デバッグ用
    #MUSIC_method_Debug(eig_val, N_ant, num_signals, resp, argm, M)
    
    est_ang = []
    # 三点比較によるピーク検出
    a, b = resp[0][0], resp[1][0]
    for i in range(1,len(argm)-1):
        c = resp[i+1][0]
        #if a < b and b > c and b > 0.2 * M:
        if a < b and b > c and b > 0.00001 * M:
            est_ang.append(argm[i])
        a, b = b, c
    return est_ang

In [161]:
def MUSIC_method_Debug(eig_val, N_ant, num_signals, resp, argm, M):
    """
    MUSIC法の状態を診断するためのデバッグ関数
    """
    print("\n" + "="*30)
    print("【MUSIC法 デバッグ情報】")
    
    # --- 1. 数値情報の出力 ---
    # 固有値の絶対値（大きい順）
    eig_abs = np.abs(eig_val)
    print(eig_abs)

    # --- 2. グラフ描画 (重要) ---
    try:
        # --- ピークのグラフだけ表示 ---
        fig, ax = plt.subplots(figsize=(8, 5))
        resp_array = np.array(resp).flatten() # 形状を1次元に統一
        
        ax.plot(argm, resp_array, label='MUSIC Spectrum')
        ax.set_title(f'MUSIC Spectrum (N_ant={N_ant}, K={num_signals})')
        ax.set_xlabel('Angle [deg]')
        ax.set_ylabel('Spectrum Power')
        ax.set_yscale('log') # スペクトルも対数で見ると弱いピークが見つけやすい
        ax.grid(True, which="both", ls="-", alpha=0.5)
        
        # 閾値の線を引く (赤の点線)
        threshold_val = 0.2 * M
        ax.axhline(y=threshold_val, color='r', linestyle='--', label='Threshold (20%)')
        
        # もし閾値を5%に下げたらどう見えるかも参考として表示 (緑の点線)
        ax.axhline(y=0.00001 * M, color='g', linestyle=':', label='Threshold (0.001%)')
        
        ax.legend()
        
        plt.tight_layout()
        plt.show()
        print("-> 診断用グラフを描画しました")
        
    except Exception as e:
        print(f"-> グラフ描画中にエラーが発生しました: {e}")
        
    print("="*30 + "\n")

In [162]:
def Root_MUSIC(Y, N_ant, num_signals=None, min_magnitude=0.8): # ← min_magnitudeのデフォルト値を調整可能に
    """
    修正版: 個数制限(K)を緩和し、閾値以上の大きさを持つ根をすべて返す
    min_magnitude: 信号根とみなす最小の絶対値（低SNR対策で少し広めにとる）
    """
    # 1. 共分散行列の計算 (変更なし)
    N_trans, N_ant_data, N_sample = Y.shape
    Y_reshaped = Y.transpose(1, 0, 2).reshape(N_ant_data, -1)
    R = (Y_reshaped @ np.conjugate(Y_reshaped.T)) / Y_reshaped.shape[1]
    
    # 2. 固有値分解 & 雑音部分空間 (変更なし)
    # ここでの num_signals は「雑音部分空間を作るための次元数」として使う
    # 推定用Kとは区別するため、呼び出し側で十分な信号数（例:5）を指定しておく
    K_subspace = 5 # 雑音空間定義用にあえて固定または引数利用
    
    eig_val, eig_vec = np.linalg.eig(R)
    idx = np.abs(eig_val).argsort()[::-1]
    eig_vec = eig_vec[:, idx]
    U_n = eig_vec[:, K_subspace:] # 上位K個以外をノイズとする
    
    # 3. 多項式の係数算出 (変更なし)
    C = U_n @ U_n.conj().T
    coeffs = np.zeros(2 * N_ant - 1, dtype=np.complex128)
    for k in range(-(N_ant - 1), N_ant):
        coeffs[k + (N_ant - 1)] = np.trace(C, offset=k)
    roots = np.roots(coeffs)
    
    # --- フィルタリング処理 ---
    # 単位円の内側のみ
    roots = roots[np.abs(roots) <= 1.0 + 1e-6]
    
    # ★ここが重要: 半径が小さい（中心に近い）根はノイズとして捨てる
    # 信号強度が弱いと、信号根も内側に入り込むため、ここの値(0.8など)で調整する
    valid_roots = roots[np.abs(roots) > min_magnitude]
    
    # ソートなどの処理
    dist_from_unit_circle = np.abs(np.abs(valid_roots) - 1.0)
    idx_sorted = dist_from_unit_circle.argsort()
    selected_roots = valid_roots[idx_sorted]
    
    # ==========================================
    # ★ここにデバッグ用コードを追加
    # ==========================================
    print(f"Total roots inside unit circle: {len(roots)}")
    print(f"Selected roots (magnitude > {min_magnitude}): {len(selected_roots)}")
    
    # グラフ描画（毎回出ると大変なので、必要に応じてコメントアウト）
    plot_roots_on_unit_circle(roots, selected_roots)
    # ==========================================

    # 5. 角度への変換
    angles_rad = np.arcsin(np.angle(selected_roots) / np.pi)
    angles_deg = np.degrees(angles_rad)
    angles_deg.sort()
    
    return angles_deg

In [163]:
def plot_roots_on_unit_circle(roots, selected_roots):
    plt.figure(figsize=(6,6))
    
    # 単位円
    t = np.linspace(0, 2*np.pi, 100)
    plt.plot(np.cos(t), np.sin(t), 'k--', alpha=0.5)
    
    # 全ての根
    plt.scatter(np.real(roots), np.imag(roots), marker='x', color='gray', label='All Roots')
    
    # 選択された根
    plt.scatter(np.real(selected_roots), np.imag(selected_roots), marker='o', color='red', label='Selected')
    
    plt.xlim(-1.2, 1.2)
    plt.ylim(-1.2, 1.2)
    plt.grid()
    plt.legend()
    plt.title("Root Constellation")
    plt.show()

In [164]:
def get_spatial_power_at_angle(Y, angle_deg, N_ant):
    """
    指定した角度における空間電力を計算する関数
    (Beamformer法によるパワー推定)
    """
    # 共分散行列 R の計算
    N_trans, N_ant_data, N_sample = Y.shape
    Y_reshaped = Y.transpose(1, 0, 2).reshape(N_ant_data, -1)
    R = (Y_reshaped @ np.conjugate(Y_reshaped.T)) / Y_reshaped.shape[1]
    
    # ステアリングベクトル
    theta_rad = angle_deg * np.pi / 180.0
    a = stevec(N_ant, theta_rad)
    
    # 電力 P = a^H * R * a
    # (実際の信号強度に比例する値が得られます)
    power = np.abs(a.H @ R @ a)[0,0]
    
    return power

In [165]:
def FFT(N_trans, N_ant, rx_sample, Y, tx, N_sample, est_ang, sym_duration, lam, Tc, env):
    # ドップラー処理
    Dopp_dft = dft(N_trans)/np.sqrt(N_trans)
    Y_dft = np.zeros((N_trans, N_ant, rx_sample), dtype=np.complex128)
    for i in range(N_ant):
        Y_dft[:,i,:] = Dopp_dft@Y[:,i,:]

    final_objects_all = []
    rd_maps = []

    P_range = np.arange(190)+620

    # FFTによる位置推定処理
    tx_arr = np.array(tx) 
    n_fft = 2**(int(rx_sample + N_sample).bit_length())
    
    # 窓関数を掛けずにFFTする
    TX_F_all = fft(tx_arr, n=n_fft, axis=1) 
    Y_F_all = fft(Y_dft, n=n_fft, axis=2)

    for i in range(len(est_ang)):
        current_angle = est_ang[i]
        ang_rad = current_angle * np.pi / 180
        
        # ステアリングベクトル (N_ant, 1)
        s_vec = np.array(stevec(N_ant, ang_rad))
        
        # --- 周波数領域でのビームフォーミング ---
        
        # 受信信号の合成: Y_combined = sum(Y * s)
        # s_vec を (1, N_ant, 1) に変形して放送
        # Y_F_all: (N_trans, N_ant, n_fft)
        # 重み付け和をとってアンテナ次元を潰す -> (N_trans, n_fft)
        # 修正前
        w_vec = np.conjugate(s_vec).reshape(1, N_ant, 1)
        Y_F_combined = np.sum(Y_F_all * w_vec, axis=1)
        # 修正後
        w_vec = s_vec.reshape(1, N_ant, 1) 
        Y_F_combined = np.sum(Y_F_all * w_vec, axis=1)

        # 送信信号(レプリカ)の合成: X_combined = sum(X * s*)
        # 元の数式 g = s* s^H X より、Xにかかる重みは s^H (つまり sの共役)
        # TX_F_all: (N_ant, n_fft)
        s_vec_conj_reshape = np.conjugate(s_vec).reshape(N_ant, 1)
        TX_F_combined = np.sum(TX_F_all * s_vec_conj_reshape, axis=0) # -> (n_fft,)

        # --- 相関演算 (Correlation) ---
        # Correlation = IFFT( FFT(Y) * conj(FFT(X)) )
        # Broadcasting: (N_trans, n_fft) * (n_fft,)
        CORR_f = Y_F_combined * np.conjugate(TX_F_combined)
        
        # 時間領域に戻す
        corr_time = ifft(CORR_f, axis=1)

        # --- 必要な範囲を切り出し ---
        rdresp_single = corr_time[:, P_range]

        # --- 結果保存 ---
        rd_maps.append(rdresp_single.copy())
        rd_maps.append(i)
        
    return rd_maps

In [232]:
def CFAR(rd_maps, est_ang, N_trans, sym_duration, lam, Tc, env):
    detected_objects = []

    for k in range(0, len(rd_maps), 2):
        rdresp_single = rd_maps[k]
        angle = rd_maps[k+1]
        # 1. CFAR実行
        # radar_inの作成 (Batch, Channel, Height, Width)
        radar_in = torch.tensor(np.abs(rdresp_single)).unsqueeze(0).unsqueeze(0)
        detections_bool = ca_cfar_improved(radar_in, convert_from_db=False)

        # 2. dBマップの準備 (Thresholder用)
        radar_db = 20 * torch.log10(radar_in[0,0])

        # 3. 閾値判定 (速度方向も見てフィルタリング)
        clean_targets = extract_peaks_scipy(radar_db, detections_bool[0,0], kernel_size=3, tar_thres=-24)

        # 5. 結果の保存
        if len(clean_targets) > 0:
            print(f"角度 {est_ang[angle]:.2f} 度 のマップからの検出:")
            for tgt in clean_targets:
                d_idx, r_idx, power = tgt
                
                # 物理量への変換 (別途 sym_duration, lam の定義が必要)
                r_est, v_est, x_est, y_est = convert_to_physical(
                    d_idx, r_idx, N_trans, sym_duration, lam, est_ang[angle], idx_to_xy, Tc, env
                )
                #detected_objects.append([r_est, v_est, current_angle, x_est, y_est]) # v_estを追加
                # 【変更点】 power もリストに追加します (末尾に追加)
                detected_objects.append([r_est, est_ang[angle], x_est, y_est, v_est, power])
                print(f"  -> Index: ({d_idx}, {r_idx}), Power: {power:.1f}dB, Range: {r_est:.2f}m, Vel: {v_est:.2f}m/s")
        else:
            print("ターゲット不検出")
        # --- ここまで ---
    # 【追加】 ここで全角度の検出結果をまとめてクリーニングします
    #final_objects = remove_ghosts_across_angles(detected_objects)    
    #final_objects = remove_ghosts_improved(detected_objects)
    #return final_objects
    return detected_objects

In [223]:
def remove_ghosts_improved(all_detections, range_tol=0.4, ang_tol=0.01, vel_tol=11.0, power_ratio_thres_db=2.0, dist_xy_tol=2.0):
    """
    物理的な重複判定とサイドローブ除去を区別して処理する関数
    
    Args:
        all_detections: [[range, angle, x, y, vel, power], ...]
        range_tol: 同一物体とみなす距離差 [m]
        ang_tol: 同一物体とみなす角度差 [deg]
        vel_tol: 同一物体とみなす速度差 [m/s]
        power_ratio_thres_db: 角度が離れている場合、このdB以上弱ければゴーストとみなす [dB]
        dist_xy_tol: XY座標上での距離許容値 [m] (物理的な近さ)
    """
    if not all_detections:
        return []
    
    # 角度ごとの検出数を記録
    angle_group_counts = {}
    for det in all_detections:
        angle = det[1]
        # 角度を丸めてグループ化 (ang_tol の半分程度の精度で)
        angle_key = round(angle / (ang_tol * 0.5)) * (ang_tol * 0.5)
        angle_group_counts[angle_key] = angle_group_counts.get(angle_key, 0) + 1

    def get_angle_group_count(angle):
        """この角度が属するグループの検出数を返す"""
        angle_key = round(angle / (ang_tol * 0.5)) * (ang_tol * 0.5)
        return angle_group_counts.get(angle_key, 1)

    # 1. パワーが強い順に並べ替える (index 5 = power)
    sorted_dets = sorted(all_detections, key=lambda x: x[5], reverse=True)
    
    final_unique_objects = []
    
    while sorted_dets:
        print("Debug")
        # 最も強い検出を取り出す（これが「本物」の可能性が最も高い）
        best = sorted_dets.pop(0)
        final_unique_objects.append(best)
        
        remaining = []
        for other in sorted_dets:
            # 各差分を計算
            r_diff = abs(best[0] - other[0])
            a_diff = abs(best[1] - other[1])
            v_diff = abs(best[4] - other[4])
            
            # XY座標での距離 (物理的に同じ場所か？)
            # best[2], best[3] が X, Y
            xy_dist = np.sqrt((best[2] - other[2])**2 + (best[3] - other[3])**2)
            
            # パワー差 (dB)
            p_diff = best[5] - other[5] # sorted済みなので必ず正の値

            # --- 判定ロジック ---

            # A. 「完全に同一の物体」の結合
            # 距離・角度・速度がすべて近い、またはXY座標が非常に近い場合
            is_same_target = (r_diff < range_tol and a_diff < ang_tol and v_diff < vel_tol) or (xy_dist < dist_xy_tol)
            is_same_target = (r_diff < range_tol and v_diff < vel_tol)

            # B. 「サイドローブ（ゴースト）」の判定
            # 距離と速度は同じだが、角度が違う場合。
            # ただし、パワーが「圧倒的に弱い（power_ratio_thres_db以上差がある）」場合のみゴーストとする。
            # ※ パワー差が小さいなら、それは並走している別の車両や自転車の可能性が高い。
            is_sidelobe = (r_diff < range_tol and v_diff < vel_tol) and (p_diff < power_ratio_thres_db)
            is_sidelobe = (a_diff < ang_tol) and (p_diff < power_ratio_thres_db)
            
            other_angle_count = get_angle_group_count(other[1])
            if other_angle_count == 1:
                is_sidelobe = False
                is_same_target = False
            # どちらにも該当しなければ、リストに残す（次のループで採用される候補）
            if is_same_target:
                print("同一物体として認識されたため除去")
                print(other)
                continue 
            elif is_sidelobe:
                print("サイドローブとして認識されたため除去")
                continue
            else:
                remaining.append(other)
        
        sorted_dets = remaining

    return final_unique_objects

In [168]:
def ca_cfar_improved(x_db, n_train=(1,7), n_guard=(1,5), pfa=1e-6, convert_from_db=False):
    """
    2次元 CA-CFAR (Cell Averaging CFAR)
    x_db: (B, 1, H, W) 入力マップ (通常はdB)
    戻り値: (B, 1, H, W) のBool型テンソル (Trueが検出位置)
    """
    # 修正
    n_guard = (1,10)
    pfa = 1e-4

    if convert_from_db:
        x = 10 ** (x_db / 10.0)
    else:
        x = x_db

    B, C, H, W = x.shape
    th, tw = n_train
    gh, gw = n_guard

    # カーネル作成 (ガードセルを0、参照セルを1にする)
    kh = 2 * (th + gh) + 1
    kw = 2 * (tw + gw) + 1
    kernel = torch.ones((1, 1, kh, kw), device=x.device, dtype=x.dtype)

    #kernel[:, :, th-gh:th+gh+1, tw-gw:tw+gw+1] = 0
    # 修正案
    kernel[:, :, th : th + 2*gh + 1, tw : tw + 2*gw + 1] = 0

    N_train = kernel.sum().item()
    
    # 閾値係数 alpha の計算
    alpha = N_train * (pfa ** (-1.0 / N_train) - 1.0)

    # 修正案
    # ドップラー方向(H)は「circular」(循環)、レンジ方向(W)は「replicate」(端の値を複製)したい場合
    # PyTorchのF.padは一度に1つのモードしか指定できないため、2段階で適用します

    # 1. レンジ方向（左右）のパディング（非循環）
    pad_w = kw // 2
    x_pad_w = F.pad(x, (pad_w, pad_w, 0, 0), mode="replicate") # または "constant"

    # 2. ドップラー方向（上下）のパディング（循環：上端と下端をつなげる）
    pad_h = kh // 2
    x_pad = F.pad(x_pad_w, (0, 0, pad_h, pad_h), mode="circular")
    
    train_sum = F.conv2d(x_pad, kernel)
    noise_est = train_sum / N_train

    threshold_map = alpha * noise_est
    detections = x > threshold_map
    
    return detections

In [169]:
def extract_peaks_scipy(radar_db, detections_bool, kernel_size=5, tar_thres=-25):
    """
    SciPyを用いた可読性の高いNMS（局所最大値検出）。
    既存の thresholder_improved と cleansing_improved を代替します。
    
    Args:
        radar_db (Tensor or ndarray): dB単位のRDマップ
        detections_bool (Tensor or ndarray): CFARの検出結果マスク
        kernel_size (int): 近傍とみなす範囲（奇数推奨, 例: 5なら5x5マス）
        tar_thres (float): パワーの閾値
        
    Returns:
        list: [[dop_idx, rng_idx, power], ...] 
        (後段の処理との互換性を維持したリスト形式)
    """
    # 1. データ型をNumPyに統一 (Tensorが来ても対応)
    if hasattr(radar_db, 'cpu'):
        img = radar_db.cpu().numpy()
    else:
        img = radar_db
        
    if hasattr(detections_bool, 'cpu'):
        mask = detections_bool.cpu().numpy()
    else:
        mask = detections_bool

    # 2. Maximum Filter で「近傍の最大値」を計算
    # これが「その範囲内で一番強い値」のマップになります
    local_max = maximum_filter(img, size=kernel_size, mode='constant', cval=-999)

    # 3. ピークの特定 (Boolean Indexing)
    # 条件A: CFARで検出されている
    # 条件B: 自分の値が近傍最大値と一致する (つまり自分がピーク)
    # 条件C: 閾値を超えている
    is_peak = mask & (img == local_max) & (img > tar_thres)

    # 4. インデックスの抽出とリスト化
    # np.argwhere で True の場所の (d_idx, r_idx) を一括取得
    peak_indices = np.argwhere(is_peak)
    
    results = []
    for d_idx, r_idx in peak_indices:
        power = float(img[d_idx, r_idx])
        results.append([d_idx, r_idx, power])
    
    # パワー順にソート (既存処理との整合性のため)
    results.sort(key=lambda x: x[2], reverse=True)
    
    return results

In [170]:
def remove_ghosts_across_angles(all_detections, range_tol=2.0, ang_tol=1.0, vel_tol=0.5):
    """
    異なる角度で検出された同一物体（サイドローブ）を統合する関数
    all_detections: [[range, vel, angle, x, y, power], ...] のリスト
    range_tol: 同一物体とみなす距離差 [m]
    vel_tol:   同一物体とみなす速度差 [m/s]
    """
    if not all_detections:
        return []

    # 1. パワーが強い順に並べ替える (一番後ろの要素がpowerと仮定)
    # x[5] が power です
    sorted_dets = sorted(all_detections, key=lambda x: x[5], reverse=True)
    
    final_unique_objects = []
    
    while sorted_dets:
        # 最も強い検出を取り出す（これが「本物」の可能性が高い）
        best = sorted_dets.pop(0)
        final_unique_objects.append(best)
        
        # 残りのリストから、「距離と角度がほぼ同じ」ものを全て削除する
        # （これらはサイドローブ＝ゴーストとみなす）
        remaining = []
        for other in sorted_dets:
            r_diff = abs(best[0] - other[0])
            a_diff = abs(best[1] - other[1])
            v_diff = abs(best[4] - other[4])
            
            # 距離と角度が近ければ、それは同じ物体のサイドローブなのでリストに残さない（削除）
            if r_diff < range_tol and a_diff < ang_tol:
                continue 
            # 距離と速度が近ければ、それは同じ物体のサイドローブなのでリストに残さない（削除）
            if r_diff < range_tol and v_diff < vel_tol:
                continue
            remaining.append(other)
        
        sorted_dets = remaining

    return final_unique_objects

In [171]:
def convert_to_physical(d_idx, r_idx, N_trans, sym_duration, lam, current_angle, idx_to_xy_func, Tc, env):
    """
    インデックスを物理的な距離(m)と速度(m/s)に変換
    """
    # 速度計算
    vel_res = lam / (2.0 * sym_duration * N_trans)
    if d_idx < N_trans / 2:
        velocity = d_idx * vel_res
    else:
        velocity = (d_idx - N_trans) * vel_res
        
    # 距離・座標計算 (既存の関数を利用)
    x, y, r = idx_to_xy_func(r_idx, current_angle, Tc, env)
    
    return r, velocity, x, y

In [172]:
def stevec(N_ant, angle):
    # 入力された角度に対するステアリングベクトルの計算
    resp = (np.arange(N_ant)-N_ant*0.5+0.5).reshape([-1,1])
    resp = np.exp(1j*resp*np.pi*np.sin(angle))
    return np.matrix(resp)

In [173]:
def idx_to_xy(range_idx, angle_deg, Tc, env):
    """
    range_idx: cy_idxs_pred の値 (例: 496)
    angle_deg: MUSIC法で推定した角度 [度] (例: est_ang[0])
    """
    p_bs = np.array([250, -18, 50])
    refangle = np.arctan(33/265)/2
    angle_offset = - 0.00005
    refangle += angle_offset
    offset = 620
    # (1) 距離の計算 [m]
    # サンプル番号 = インデックス + オフセット
    sample_idx = range_idx + offset
    # 往復時間 = サンプル番号 * Tc
    # 距離 = 往復時間 * c / 2
    R = (sample_idx * Tc * env.c) / 2

    # 三次元距離を二次元距離に変換
    height_diff = p_bs[2] - 1
    if R > height_diff:
        R_horizontal = np.sqrt(R**2 - height_diff**2)
    else:
        R_horizontal = 0

    # (2) 角度の計算 [rad]
    # 推定角度をラジアンに変換し、基準角度(refangle)を足す
    # anglecalの逆演算: cy_ang = arctan(...) - refangle なので
    # arctan(...) = cy_ang + refangle
    theta_world = np.deg2rad(angle_deg) + refangle

    # (3) XY座標の計算 [m]
    # 基地局(p_bs)からの相対位置
    # cy_denum (dx) = R * cos(theta)
    # cy_nu (dy) = R * sin(theta)
    dx = R_horizontal * np.cos(theta_world)
    dy = R_horizontal * np.sin(theta_world)

    # 世界座標（絶対座標）に変換
    # cy_denum = bs[0] - cy[0]  =>  cy[0] = bs[0] - dx
    # cy_nu    = cy[1] - bs[1]  =>  cy[1] = bs[1] + dy
    x = p_bs[0] - dx
    y = p_bs[1] + dy

    return x, y, R

In [174]:
def RD_MAP(rd_maps, frame_id, T_symbol, Tc, save_dir="./rd_maps/"):
    l_speed = 299792458 
    f_carrier = 28 * 1e+9

    lam = l_speed / f_carrier
    
    
    # ディレクトリがなければ作成
    os.makedirs(save_dir, exist_ok=True)
    
    rdresp_list = np.array(rd_maps[::2])
    #idx = np.array(rd_maps[1::2])
    
    # 距離分解能 (m)
    range_res = l_speed * Tc / 2
    
    # 横軸 (距離):
    start_sample_idx = 620
    num_range_bins = 190

    r_min = start_sample_idx * range_res
    r_max = (start_sample_idx + num_range_bins) * range_res

    v_range = lam / (2 * T_symbol)  # 最大速度範囲

    v_min_val = -v_range / 2
    v_max_val = v_range / 2
    
    i = 0

    for rd_resp in rdresp_list:
        #output = rd_resp[:100,:] + 1e-10
        #output = rd_resp + 1e-10
        #output = np.fft.fftshift(rd_resp, axes=0) + 1e-10
        # 1. ゼロ周波数を中央にシフト (縦軸方向)
        output_shifted = np.fft.fftshift(rd_resp, axes=0)
        # 2. dB変換 (対数)
        # 【重要】絶対値をとってから微小値を足す (0除算防止 & ノイズフロア確保)
        output_db = 20 * np.log10(np.abs(output_shifted) + 1e-20)

        #plt.imshow(20*np.log10(np.abs(output)), aspect='auto', interpolation='nearest',
        #     extent = [r_min, r_max, v_min_val, v_max_val], origin='lower', vmin=-30, vmax = -10)
        plt.imshow(output_db, 
                   aspect='auto', 
                   interpolation='nearest', # ドットを潰さない
                   extent=[r_min, r_max, v_min_val, v_max_val], 
                   origin='lower', 
                   vmin=-30,   # 引数で指定した値を使用
                   vmax=10,   # 引数で指定した値を使用
        )
        plt.xlabel('Range [m]')
        plt.ylabel('Velocity [m/s]') # 注: DFTのシフトをしていないため、軸の解釈に注意が必要
        plt.colorbar(label='Power [dB]')
        plt.title('Range-Doppler Map')
        plt.grid(False) # 任意
        plt.xlim(r_min,r_max)
        plt.ylim(-30,30)
        # --- 保存処理 ---
        # ファイル名を決定 (例: ./rd_maps/rd_map_12345.png)
        save_path = os.path.join(save_dir, f"rd_map_{frame_id}_for_{i}.png")
        plt.savefig(save_path)
        print(f"Saved: {save_path}")
        
        # --- クローズ処理 ---
        # show() ではなく close() することで、画面表示せずバックグラウンドで処理できます
        # 表示もしたい場合は plt.show() の前に plt.savefig() を置いてください
        plt.close()
        i += 1
    
    # 保存する
    

In [227]:
def print_true_values(num_cy, num_ve, num_rp, phys_quantities, real_cy_coordinates, real_vel_coordinates, real_rp_coordinates,
                      real_cy_coordinates_all, real_cy_ranges_all, real_cy_angles_all, real_cy_velocities_all,
                      real_ve_coordinates_all, real_ve_ranges_all, real_ve_angles_all, real_ve_velocities_all):
    if num_cy > 0:
        print("Cyclist True Position:", real_cy_coordinates)
        print("Cyclist True Range:", phys_quantities["range"]["cyclists"][0])
        print("Cyclist True Angle:", math.degrees(phys_quantities["angle"]["cyclists"][0]))
        a = math.degrees(phys_quantities["angle"]["cyclists"][0])
        real_cy_coordinates_all.append(real_cy_coordinates)
        real_cy_ranges_all.append(phys_quantities["range"]["cyclists"][0])
        real_cy_angles_all.append(phys_quantities["angle"]["cyclists"][0])
        real_cy_velocities_all.append(phys_quantities["relative_velocity"]["cyclists"][0])
    if num_ve > 0:
        print("Vehicle True Position:", real_vel_coordinates)
        print("Vehicle True Range:", phys_quantities["range"]["vehicles"][0])
        print("Vehicle True Angle:", math.degrees(phys_quantities["angle"]["vehicles"][0]))
        b= math.degrees(phys_quantities["angle"]["vehicles"][0])
        real_ve_coordinates_all.append(real_vel_coordinates)
        real_ve_ranges_all.append(phys_quantities["range"]["vehicles"][0])
        real_ve_angles_all.append(phys_quantities["angle"]["vehicles"][0])
        real_ve_velocities_all.append(phys_quantities["relative_velocity"]["vehicles"][0])
    if num_rp > 0:
        print("Ramp Post True Position:", real_rp_coordinates)
        print("Ramp Post True Range:", phys_quantities["range"]["ramposts"])
        print("Ramp Post True Angle:", math.degrees(phys_quantities["angle"]["ramposts"]))

    #print('angle difference:', np.abs(a-b))
    
    return

In [176]:
def main_simulation(i):
    # --- 保存先ディレクトリの設定 ---
    base_dir = "./montecarlo"
    rd_map_dir = os.path.join(base_dir, "rd_maps")
    
    os.makedirs(rd_map_dir, exist_ok=True)
    
    # 実行用設定
    T_symbol,f_carrier,Tc,N_ant,BW,BW_sub,N_sample,rx_sample, mu, l_speed, lam, env = Radar_setting()
    N_chirp = 89
    print("DEBUG: ", T_symbol * N_chirp)
    N_trans = N_chirp
    tx = env.tx()
    p_bs, num_cy, num_ve, num_rp, start_cy_idx, start_ve_idx, end_cy_idx, end_ve_idx, cy_v_value, ve_v_value, cy_interval, ve_interval = Sim_Setting()
    cy_idx, ve_idx = start_cy_idx, start_ve_idx
    measurement_history = []
    frame_count = 0
    
    # 画像フォルダの中身を削除 (必要に応じてコメントアウト)
    # 毎回削除すると、前の回の画像が消えます。
    derectory_deletor(rd_map_dir)
    
    T_sensing = N_chirp * T_symbol + 813 * Tc
    
    real_cy_coordinates_all, real_cy_ranges_all, real_cy_angles_all, real_cy_velocities_all, real_ve_coordinates_all, real_ve_ranges_all, real_ve_angles_all, real_ve_velocities_all = [], [], [], [], [], [], [], []

    while cy_idx <= end_cy_idx and ve_idx <= end_ve_idx:
        x_pos = list(np.arange(-450, -149)/10)
        
        Y, phys_quantities, real_cy_coordinates, real_vel_coordinates, real_rp_coordinates = get_snapshot_data(cy_idx,ve_idx,x_pos,p_bs, tx, cy_v_value, ve_v_value, num_cy, num_ve, num_rp, T_symbol, N_trans, Tc, env)
        print('使っているインデックス:', cy_idx, ve_idx)
        print_true_values(num_cy, num_ve, num_rp, phys_quantities, real_cy_coordinates, real_vel_coordinates, real_rp_coordinates,
                      real_cy_coordinates_all, real_cy_ranges_all, real_cy_angles_all, real_cy_velocities_all,
                      real_ve_coordinates_all, real_ve_ranges_all, real_ve_angles_all, real_ve_velocities_all) 
        
        est_ang = MUSIC_method_improved(Y, N_ant, num_cy + num_ve + num_rp)
        
        """
        # ==========================================================
        # 【修正版】Root-MUSIC + パワー閾値判定による検出ロジック
        # ==========================================================
        
        # 1. Root-MUSICで候補となる角度を算出
        #    ターゲット数(2)より多めのKを設定し、自転車を取りこぼさないようにする
        #    (自動車のサイドローブやノイズも候補に含まれるが、後段でフィルタリングする)
        K_search = 5  
        candidate_angles = Root_MUSIC(Y, N_ant, num_signals=K_search)
        
        est_ang = []
        
        # 2. パワー閾値の設定 [重要]
        #    実験結果に基づき、ノイズフロア(~2.8e-11)より大きく、
        #    自転車+自動車漏れ(~8.0e-11)を拾えるラインに設定
        POWER_THRESHOLD = 2.0e-11 

        # 3. 候補ごとの検証
        for ang in candidate_angles:
            # その角度における空間電力(ビームフォーマー出力)を計算
            
            p = get_spatial_power_at_angle(Y, ang, N_ant)

            # 閾値を超えている場合のみ「物体」として採用
            if p > POWER_THRESHOLD:
                est_ang.append(ang)
                # デバッグ用: 検出された角度とパワーを表示
                # print(f"  [Accept] Angle: {ang:.2f} deg, Power: {p:.2e}")
            else:
                pass
                # デバッグ用: 棄却された角度とパワーを表示
                # print(f"  [Reject] Angle: {ang:.2f} deg, Power: {p:.2e}")

        # リストをソートして出力
        est_ang.sort()
        
        # 万が一何も検出されなかった場合の処理（空リストのままでOKなら何もしない）
        if len(est_ang) == 0:
            pass
            
        # ==========================================================    
        """

        rd_maps = FFT(N_trans, N_ant, rx_sample, Y, tx, N_sample, est_ang, T_symbol, lam, Tc, env)

        detected_objects = CFAR(rd_maps, est_ang, N_trans, T_symbol, lam, Tc, env)
        
        # 画像保存 (シミュレーション番号 i をファイル名に含めると区別可能です)
        # ただし derectory_deletor が有効だと消えてしまうので注意
        RD_MAP(rd_maps, frame_count, T_symbol, Tc, save_dir=rd_map_dir)
        
        if len(detected_objects) > 0:
            for obj in detected_objects:
                measurement_history.append({
                    "Time": frame_count,
                    "Range": obj[0],
                    "Angle": obj[1],
                    "X": obj[2],
                    "Y": obj[3],
                    "Velocity": obj[4],
                    "Power": obj[5]
                })
        else:
            measurement_history.append({
                "Time": frame_count,
                "Range": np.nan,
                "Angle": np.nan,
                "X": np.nan,
                "Y": np.nan,
                "Velocity": np.nan,
                "Power": np.nan
            })

        angle_list = []
        power_list = []

        for obj in detected_objects:
            if obj[1] in angle_list:
                continue
            else:
                angle_list.append(obj[1])
            if obj[5] in power_list:
                continue
            else:
                power_list.append(obj[5])
        # --- 角度分解に成功したかを表示 ---
        if len(est_ang) > 1:
            print(len(est_ang))
            print(len(angle_list))
            if len(est_ang) == len(angle_list):
                print("成功，強度差は ->", np.abs(power_list[0]-power_list[1]))
                for obj in detected_objects:
                    print(obj)
            else:
                print("処理上で何らかの問題が発生")
        else:
            print("失敗")
        
        cy_idx += cy_interval
        ve_idx += ve_interval
        frame_count += 1
        
    # --- CSV保存 (番号付き) ---
    df_measurements = pd.DataFrame(measurement_history)
    csv_filename = os.path.join(base_dir, f"measurements_{i}.csv") # ここを修正
    df_measurements.to_csv(csv_filename, index=False)

    print("\nData generation complete.")
    print(f"Saved '{csv_filename}'.")

    real_positions = []
    csv_filename_true = os.path.join(base_dir, f"true_information_{i}.csv") # ここを修正
    
    # 【重要修正】 変数名を i から k に変更 (引数 i との衝突回避)
    for k in range(max(len(real_cy_coordinates_all), len(real_ve_coordinates_all))):
        if num_cy > 0:
            real_positions.append({
                "Time": k,
                "target_id":0,
                "X": real_cy_coordinates_all[k][0][0],
                "Y": real_cy_coordinates_all[k][0][1],
                "Range": real_cy_ranges_all[k],
                "Angle": real_cy_angles_all[k],
                "Velocity": real_cy_velocities_all[k],
            })

        if num_ve > 0:
            real_positions.append({
                "Time": k,
                "target_id":1,
                "X": real_ve_coordinates_all[k][0][0],
                "Y": real_ve_coordinates_all[k][0][1],
                "Range": real_ve_ranges_all[k],
                "Angle": real_ve_angles_all[k],
                "Velocity": real_ve_velocities_all[k]
            })
            
    df_real_positions = pd.DataFrame(real_positions)
    df_real_positions.to_csv(csv_filename_true, index=False)
    print(f"Saved true positions to '{csv_filename_true}'.")

In [233]:
def montecarlo_simulation():
    np.random.seed(42)  # 再現性のためのシード設定
    num_simulations = 1
    for i in range(num_simulations):
        print(f"\n--- Simulation Run {i+1}/{num_simulations} ---")
        main_simulation(i)

montecarlo_simulation()


--- Simulation Run 1/1 ---
DEBUG:  0.000496175
使っているインデックス: 150 100
Cyclist True Position: [array([-30.,  15.])]
Cyclist True Range: 286.1642884777903
Cyclist True Angle: 3.1753717668567645
Vehicle True Position: [array([-35. ,   7.5])]
Vehicle True Range: 290.30372026551777
Vehicle True Angle: 1.5665124267790125
角度 1.57 度 のマップからの検出:
  -> Index: (1, 136), Power: -8.2dB, Range: 288.40m, Vel: 10.79m/s
  -> Index: (1, 131), Power: -21.3dB, Range: 286.50m, Vel: 10.79m/s
角度 3.17 度 のマップからの検出:
  -> Index: (1, 136), Power: -9.5dB, Range: 288.40m, Vel: 10.79m/s
  -> Index: (1, 131), Power: -19.8dB, Range: 286.50m, Vel: 10.79m/s
Saved: ./montecarlo\rd_maps\rd_map_0_for_0.png
Saved: ./montecarlo\rd_maps\rd_map_0_for_1.png
2
2
成功，強度差は -> 1.269108621673377
[np.float64(288.40334252058005), np.float64(1.57), np.float64(-33.07790286927559), np.float64(7.345391773969226), np.float64(10.789412505668365), -8.211467750162502]
[np.float64(286.49591300655504), np.float64(1.57), np.float64(-31.1498535165371

In [178]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

def plot_range_power_with_miss_counts(file_path):
    # 1. CSVファイルを読み込む
    try:
        df = pd.read_csv(file_path)
    except FileNotFoundError:
        print(f"エラー: ファイル '{file_path}' が見つかりませんでした。")
        return

    # 必要な列の確認
    required_cols = ['Time', 'Range', 'Power']
    if not all(col in df.columns for col in required_cols):
        print(f"エラー: 必要な列 {required_cols} がCSVに含まれていません。")
        return

    # 2. データの整理と欠損の特定
    # Timeをインデックスに設定
    df = df.sort_values('Time')
    df = df.set_index('Time')

    # 時間ステップ（間隔）を自動判定（最頻値を使用）
    # データが1行しかない場合などは1と仮定
    if len(df) > 1:
        time_diffs = df.index.to_series().diff().dropna()
        dt = time_diffs.mode()[0]  # 最も頻出する間隔をステップとする
    else:
        dt = 1 

    # 完全な時間の並びを作成（最初から最後までの等間隔な時刻）
    full_time_idx = range(int(df.index.min()), int(df.index.max()) + int(dt), int(dt))
    
    # データを再構築（欠損していた時刻にはNaNが入る）
    df_reindexed = df.reindex(full_time_idx)

    # 検出漏れの判定（RangeまたはPowerがNaNの行）
    is_missed = df_reindexed['Range'].isna() | df_reindexed['Power'].isna()
    miss_count = is_missed.sum()

    print(f"検出漏れ回数: {miss_count} 回")

    # 3. プロット用のデータ準備
    # 検出漏れ箇所をプロットするために補間（Interpolate）を行う
    # Time（インデックス）に基づいて線形補間
    df_interp = df_reindexed.interpolate(method='index')

    # プロット開始
    plt.figure(figsize=(10, 6))

    # (A) 検出成功データのプロット（青い線と丸）
    # 欠損箇所で線が切れるように、補間していない元のdf_reindexedを使用
    plt.plot(df_reindexed['Range'], df_reindexed['Power'], 
             'o-', color='blue', alpha=0.7, label='Detected Signal')

    # (B) 検出漏れ箇所のプロット（赤い×）
    # 補間データのうち、検出漏れだった箇所だけを抽出してプロット
    if miss_count > 0:
        missed_data = df_interp[is_missed]
        plt.scatter(missed_data['Range'], missed_data['Power'], 
                    color='red', marker='x', s=100, zorder=5, 
                    label=f'Missed Detection ({miss_count} times)')
        
        # オプション: 検出漏れ箇所を赤い点線で結ぶ場合
        # plt.plot(df_interp['Range'], df_interp['Power'], 'r:', alpha=0.4)

    # 4. グラフの装飾
    plt.title(f'Range vs Power (Total Misses: {miss_count})')
    plt.xlabel('Distance (Range) [m]')
    plt.ylabel('Intensity (Power) [dB]')
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.legend()
    plt.ylim(-28, -7)
    # 表示
    plt.show()

if __name__ == "__main__":
    plot_range_power_with_miss_counts('montecarlo/measurements_0.csv')

ValueError: cannot reindex on an axis with duplicate labels